In [28]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import  DataLoader, Dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

In [29]:
full_dataset = 'Datasets'
train_path = 'Datasets/train'
test_path ='Datasets/test'

In [30]:
# Transformations for training data (with augmentation)
train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1), # Add grayscale transformation
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5],  # Placeholder values - will be updated after recalculating mean/std
                         [0.5])
])

# Transformations for validation data (no augmentation)
val_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1), # Add grayscale transformation
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5],  # Placeholder values - will be updated after recalculating mean/std
                         [0.5])
])

In [31]:
# Dataset dari struktur folder
train_set = datasets.ImageFolder(root=train_path, transform=train_transform)
test_set  = datasets.ImageFolder(root=test_path,  transform=val_transform)

In [32]:
# Split train_set → 70% train, 20% val
total_test = len(test_set)
test_size  = int(0.5 * total_test)  # 50% test
val_size    = total_test - test_size  # 50% val

test_dataset, val_dataset = random_split(test_set, [test_size, val_size])

# train_set tetap dipakai sebagai train_dataset
train_dataset = train_set

In [33]:
# Loader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

In [34]:

print(f"Train samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

Train samples: 28709
Validation samples: 3589
Test samples: 3589


In [35]:
class AnimalCNN(nn.Module):
    def __init__(self):
        super(AnimalCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool  = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1   = nn.Linear(128 * 16 * 16, 256)
        self.fc2   = nn.Linear(256, 7)  # 7 class: emosi
        self.dropout = nn.Dropout(0.25)


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 32x112x112 → 32x56x56
        x = self.pool(F.relu(self.conv2(x)))  # 64x56x56  → 64x28x28
        x = self.pool(F.relu(self.conv3(x)))  # 128x28x28 → 128x14x14
        # x = x.view(-1, 128 * 14 * 14)
        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Initialize model and move to device
model = AnimalCNN().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

cpu


In [ ]:
def train_model(model, train_loader, val_loader, test_loader, criterion, optimizer, epochs=10, device='cpu'):
    model.to(device)
    best_val_acc = 0.0

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        print("-" * 40)

        # === TRAINING PHASE ===
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total
        print(f"✅ Training   - Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")

        # === VALIDATION PHASE ===
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        val_acc = 100 * val_correct / val_total
        print(f"📊 Validation - Loss: {avg_val_loss:.4f}, Accuracy: {val_acc:.2f}%")

        # === SAVE BEST MODEL ===
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
            print("🔥 Best model disimpan: best_model.pth")

    print("\n🎯 Training selesai!")

    # === TESTING PHASE (Setelah Semua Epoch) ===
    print("\n--- Evaluasi Akhir pada Test Set ---")
    model.eval()
    test_loss = 0.0
    test_correct = 0
    test_total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    test_acc = 100 * test_correct / test_total
    print(f"\n🧪 Test Final - Loss: {avg_test_loss:.4f}, Accuracy: {test_acc:.2f}%")

    return model


In [ ]:
trained_model = train_model(model, train_loader, val_loader, test_loader, criterion, optimizer, epochs=10)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import torch

# Set model to evaluation mode
model.eval()

y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

# Access class names correctly for Subset
class_names = val_dataset.dataset.classes

In [ ]:
report = classification_report(y_true, y_pred, target_names=class_names)
print("Classification Report:\n", report)

In [ ]:
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Recreate the dataset with the updated transform
train_set_gray = datasets.ImageFolder(root=train_path, transform=train_transform)

# Recreate the DataLoader with the updated dataset
train_loader_gray = DataLoader(train_set_gray, batch_size=64, shuffle=True, num_workers=2)

# Calculate mean and std for grayscale images
def get_mean_std_gray(loader):
    mean = 0.
    std = 0.
    total_images_count = 0

    for images, _ in tqdm(loader):
        batch_samples = images.size(0)  # jumlah gambar di batch
        images = images.view(batch_samples, -1)  # reshape to (batch, pixels)
        mean += images.mean(1).sum(0)
        std += images.std(1).sum(0)
        total_images_count += batch_samples

    mean /= total_images_count
    std /= total_images_count
    return mean, std

mean_gray, std_gray = get_mean_std_gray(train_loader_gray)
print("Mean (Grayscale):", mean_gray)
print("Std (Grayscale):", std_gray)